In [1]:
import numpy as np
import tensorflow as tf
import cv2
import keras
from keras import layers
from keras import optimizers
from keras.layers import Input, Lambda, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, AlphaDropout
from keras.layers import Conv2D, SpatialDropout2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.utils import plot_model
import keras.backend as K
from keras.applications import resnet, resnet_v2, vgg16, vgg19
from keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# prepare data

(x_tr, y_tr), (x_test_norm, y_test_norm) = mnist.load_data()
x_tr = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_tr])
x_test_norm = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_test_norm])

x_tr = x_tr.reshape(60000, 56, 56, 1)
x_test_norm = x_test_norm.reshape(10000, 56, 56, 1)
y_tr = to_categorical(y_tr)
y_test_norm = to_categorical(y_test_norm)

In [6]:
# create model

base_model = resnet.ResNet50(input_shape=(56, 56, 1), include_top=False, weights=None, pooling='avg')

x = Dense(10, activation = 'softmax')(base_model.layers[-1].output)

model = Model(inputs=base_model.input, outputs=x)

In [9]:
optimizer = keras.optimizers.SGD(lr=0.0005, momentum = 0.9, nesterov = True)
model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [12]:
model.fit(x_tr, y_tr, validation_data = (x_test_norm, y_test_norm), batch_size=32, epochs=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 235s 4ms/step - loss: 0.0178 - accuracy: 0.9949 - val_loss: 0.0240 - val_accuracy: 0.9921


In [13]:
# model.save('models/resnet_99.h5')

In [15]:
model1 = load_model('models/resnet_99.h5')

In [14]:
attacktest = pd.read_csv('whitetoblack.csv', header=None)
attacktest = attacktest.to_numpy()
attacktest = attacktest.reshape(10000, 56, 56, 1)
model.evaluate(attacktest, y_test_norm)

10000/10000 [==============================] - 10s 959us/step


[0.03543710993916029, 0.9889000058174133]

In [5]:
model.evaluate(x_test_norm, y_test_norm)

10000/10000 [==============================] - 11s 1ms/step


[6.741469145965576, 0.10400000214576721]